In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time

# --- 로깅 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300

# --- JetBot 객체 초기화 ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리 함수 ---
async def handle_command(command: str, parameters: dict = None):
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")
    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left")
            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0})
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0})
            await handle_command("forward", {"speed": 0.4, "duration": 1.0})
        elif command == "stop":
            robot.stop()
        elif command == "rotate":
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3)
                duration = abs(angle) / 45.0 * 0.7
                if angle > 0:
                    robot.right(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")
        elif command == "custom_command":
            if parameters and "prompt" in parameters:
                logger.info(f"Received Custom command prompt: {parameters['prompt']}")
            else:
                logger.warning("custom_command missing prompt")
        elif command == "none":
            pass
        else:
            logger.warning(f"Unknown command: {command}")
    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")

# --- 이미지 전송 함수 ---
async def send_image(websocket):
    try:
        image = camera.value
        if image is not None:
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))
    except Exception as e:
        logger.error(f"Error sending image: {e}")

# --- 웹소켓 핸들러 ---
async def websocket_handler(websocket, path):
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        # 실시간 이미지 전송 루프
        async def image_stream():
            while True:
                await send_image(websocket)
                await asyncio.sleep(0.1)  # 10fps로 설정 (조정 가능)

        # 이미지 스트림 태스크 시작
        stream_task = asyncio.create_task(image_stream())

        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling message: {e}")

        # 클라이언트 연결 종료 시 스트림 태스크 취소
        stream_task.cancel()
    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")

# --- 메인 함수 ---
async def main():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started on port {WEBSOCKET_PORT}")
    await server.wait_closed()

# --- 실행 함수 ---
def run():
    loop = asyncio.get_event_loop()
    if loop.is_running():
        task = loop.create_task(main())
        return task
    else:
        try:
            loop.run_until_complete(main())
        except KeyboardInterrupt:
            logger.info("Shutting down...")
            robot.stop()
            camera.stop()
        finally:
            loop.close()

if __name__ == "__main__":
    try:
        run()
    except Exception as e:
        logger.error(f"Error in main execution: {e}")
    finally:
        robot.stop()
        camera.stop()

DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Reseting PCA9685 MODE1 (without SLEEP) and MODE2
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFA
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFB
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFC
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFD
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x04 to register 0x01
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Setting PWM frequency to 1600 Hz
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Estimated pre-scale: 2
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Final pre-scale: 3
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2